In [58]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import warnings
import math
import sys
from tabulate import tabulate
import csv

import CovidEM
import CovidEM2
import CovidCI

import imp
imp.reload(CovidCI)
imp.reload(CovidEM) #applies changes made to file
imp.reload(CovidEM2)

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

# print(cases)
# print(prem["can"])

In [59]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI
np.random.seed(451)

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
    k, p = params
    return np.exp(k), 1/(1+np.exp(-p))
    #return k, p

# Log-Likelihood function for a negative binomial model
def LogL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [4, 0.2]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
#print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))



The estimate is: k=4.233, p=0.086


In [60]:
np.random.seed(451)

# Define gradient and Hessian
jac = nd.Gradient(LogL)
hess = nd.Hessian(LogL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, LogL, jac, hess, alpha = 0.95,
              disp=False) # the disp argument lets the algorithm print
                         # status messages.
    
#print(result.x)
#print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
#print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
#   original_lower[1], original_upper[1]))
print("\n")
print("trans: k: ", transform_parameters(result.x)[0])
print("trans: k - upper:", transform_parameters(result.x)[0] - original_upper[0])
print("trans: k - lower:", transform_parameters(result.x)[0] - original_lower[0])
print("\n")
print("non-trans: k: ", result.x[0])
print("non-trans: k - upper: ", result.x[0] - CIs[0,0])
print("non-trans: k - lower: ", result.x[0] - CIs[0,1])

Confidence interval for k: [3.106, 5.687]


trans: k:  4.2334692165135435
trans: k - upper: -1.4535306447825596
trans: k - lower: 1.1274530109756


non-trans: k:  1.4430218026703197
non-trans: k - upper:  0.30968086046292687
non-trans: k - lower:  -0.2951610412647001


In [61]:
imp.reload(CovidEM2)

starts = 1 #number of different starting points to test
trials = 1 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 100, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    for k in range(len(theta_guess)): theta_guess[k] = math.log(theta_guess[k], math.e)
    for j in range(0, trials):
        res = optimize.basinhopping(CovidEM2.Covid_KL2, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:

100.00 %, --------  --------  -  --------  --------  ---------  --------  --------  --------  ---------  -  ---------  --------  ---------  --------  --------  --------  ---------  -  --------  --------  --------  --------  --------  -------  --------  -  ---------  -------  --------  --------  -------  --------
0.253339  0.462547  1  -1.18867  -0.42436  -0.203693  0.276034  0.106561  -1.12905  -0.377968  1  -0.328142  0.246316  -0.172576  0.130968  0.755308  -1.61294  -0.774383  1  0.097179  0.433981  0.457346  0.496305  0.543279  1.54297  -0.91415  1  0.0149168  -11.815  0.725677  0.926855  1.21733  0.316785
--------  --------  -  --------  --------  ---------  --------  --------  --------  ---------  -  ---------  --------  ---------  --------  --------  --------  ---------  -  --------  --------  --------  --------  --------  -------  --------  -  ---------  -------  --------  --------  -------  --------


In [115]:
imp.reload(CovidEM2) #applies changes made to file
np.random.seed(451)


best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081, 1.727919503, 1.734579037, 
            3.77747199,  0.71170154,  0.769347828, 0.104869899, 2.090248998, 1.98923028,  3.373799478]
for i in range(0, len(best_est)):
    best_est[i] = math.log(best_est[i])

# Initialize variables
CovidEM2.set_prem(prem)
CovidEM2.set_cases(cases_dict)
CovidEM2.set_countries(countries)


#cis_2a = CovidEM2.CI_calc2(best_res.x, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone a\n\n")

cis_2b = CovidEM2.CI_calc2(best_est, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone b\n\n")




Done a


iter   1<: x_0_d= 0.125; f_d= 2.233; jac_d= 0.507; nsteps= 5; x_d= 0.287; f_impr=-0.218; jac_impr=-0.513; f_e= 0.000; step= 0.250; radius= 0.296 - maximizing nuisance parameters
-> iter   1<: !!!found NEW MAXIMUM for x_0 of  8.334 (+ 0.313) at [-1.64234058 -1.25851116  0.39604187  0.69267139  0.73153547  0.8585906
  0.85201246  1.32708887 -0.14518022  0.24156855 -2.40231319  0.4761804
  0.2807195   1.31164877]!!!
iter   2<: x_0_d= 0.375; f_d= 2.313; jac_d= 2.107; nsteps= 3; x_d= 0.294; f_impr=-0.051; jac_impr=-2.055; f_e= 0.000; step= 0.250; radius= 0.296 - maximizing nuisance parameters
-> iter   2<: !!!found NEW MAXIMUM for x_0 of  8.414 (+ 0.393) at [-1.89234058 -1.67248146  0.09284141  0.49769733  0.67114511  0.77102115
  0.62239056  1.53462174 -0.13953011  0.67917779 -2.30646259 -0.01814302
 -0.12713489  1.66917118]!!!
iter   3<: x_0_d= 0.406; f_d= 2.369; jac_d= 3.080; nsteps= 5; x_d= 0.087; f_impr=-0.034; jac_impr=-0.548; f_e= 0.000; step= 0.062; radius= 0.088 - maxim

iter  28<: x_0_d= 5.948; f_d=-0.075; jac_d= 0.004; nsteps= 3; x_d= 0.087; f_impr= 0.045; jac_impr= 0.408; f_e= 0.000; step= 0.125; radius= 0.090 - maximizing nuisance parameters
iter  29<: x_0_d= 5.948; f_d=-0.073; jac_d= 0.003; nsteps= 3; x_d= 0.087; f_impr= 0.037; jac_impr= 0.359; f_e= 0.000; step= 0.125; radius= 0.090 - maximizing nuisance parameters
iter  30<: x_0_d= 5.948; f_d=-0.071; jac_d= 0.003; nsteps= 3; x_d= 0.087; f_impr= 0.032; jac_impr= 0.331; f_e= 0.000; step= 0.125; radius= 0.090 - maximizing nuisance parameters
iter  31<: x_0_d= 5.948; f_d=-0.070; jac_d= 0.002; nsteps= 3; x_d= 0.087; f_impr= 0.027; jac_impr= 0.311; f_e= 0.000; step= 0.125; radius= 0.090 - maximizing nuisance parameters
iter  32<: x_0_d= 5.948; f_d=-0.069; jac_d= 0.002; nsteps= 3; x_d= 0.087; f_impr= 0.024; jac_impr= 0.257; f_e= 0.000; step= 0.125; radius= 0.090 - maximizing nuisance parameters
iter  33<: x_0_d= 5.948; f_d=-0.068; jac_d= 0.002; nsteps= 3; x_d= 0.087; f_impr= 0.022; jac_impr= 0.114; f_e=

iter   1<: x_1_d= 0.062; f_d= 2.098; jac_d= 1.104; nsteps= 5; x_d= 0.286; f_impr=-0.127; jac_impr=-1.773; f_e= 0.000; step= 0.125; radius= 0.296 - maximizing nuisance parameters
-> iter   1<: !!!found NEW MAXIMUM for x_1 of  8.198 (+ 0.177) at [-1.39623992 -0.92215098  0.36760583  0.66159179  0.70506326  0.84218126
  0.78708497  1.09734284 -0.6343017   0.38227078 -2.35993568  0.50277557
  0.31552872  1.39357352]!!!
iter   2<: x_1_d= 0.070; f_d= 2.194; jac_d= 0.911; nsteps= 6; x_d= 0.056; f_impr=-0.065; jac_impr= 0.277; f_e= 0.000; step= 0.016; radius= 0.059 - maximizing nuisance parameters
-> iter   2<: !!!found NEW MAXIMUM for x_1 of  8.294 (+ 0.273) at [-1.4672844  -0.92996348  0.27948174  0.61473037  0.6966523   0.83373911
  0.76802254  1.12699767 -0.62081724  0.42387309 -2.319795    0.50543175
  0.23817613  1.53003976]!!!
iter   3<: x_1_d= 0.133; f_d= 2.256; jac_d= 0.989; nsteps= 4; x_d= 0.116; f_impr=-0.040; jac_impr=-0.118; f_e= 0.000; step= 0.062; radius= 0.117 - maximizing nuis

iter  21<: x_1_d= 0.714; f_d= 2.636; jac_d= 0.535; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.069; f_e= 0.000; step= 0.001; radius= 0.001 - maximizing nuisance parameters
-> iter  21<: !!!found NEW MAXIMUM for x_1 of  8.736 (+ 0.715) at [-1.85514402 -1.57327403  0.22783984  0.5855582   0.82974866  0.8957288
  0.62111978  1.61055573 -1.17211781  0.40785959 -1.26249719 -1.16582394
 -0.16793867  1.57016805]!!!
iter  22<: x_1_d= 0.715; f_d= 2.638; jac_d= 0.351; nsteps= 3; x_d= 0.001; f_impr=-0.001; jac_impr= 0.607; f_e= 0.000; step= 0.001; radius= 0.001 - maximizing nuisance parameters
-> iter  22<: !!!found NEW MAXIMUM for x_1 of  8.738 (+ 0.717) at [-1.85478505 -1.57425059  0.22822084  0.58582309  0.83004895  0.89640157
  0.62482386  1.61076794 -1.17208565  0.40876362 -1.26207004 -1.16581984
 -0.16778444  1.57119144]!!!
iter  23<: x_1_d= 0.715; f_d= 2.638; jac_d= 0.369; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.073; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisa

iter  41<: x_1_d= 0.724; f_d= 2.649; jac_d= 0.227; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr= 0.486; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
-> iter  41<: !!!found NEW MAXIMUM for x_1 of  8.749 (+ 0.728) at [-1.8491736  -1.58377208  0.22471252  0.58653378  0.83248368  0.90231164
  0.63320872  1.6139684  -1.17140834  0.41803149 -1.25408155 -1.16706915
 -0.16384429  1.55970198]!!!
iter  42<: x_1_d= 0.725; f_d= 2.649; jac_d= 0.439; nsteps= 4; x_d= 0.001; f_impr=-0.000; jac_impr=-0.953; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
-> iter  42<: !!!found NEW MAXIMUM for x_1 of  8.750 (+ 0.729) at [-1.84875919 -1.58426036  0.22407131  0.58682366  0.83280144  0.90253407
  0.6361781   1.61428665 -1.17136081  0.41953948 -1.25351305 -1.16708026
 -0.16379866  1.55997349]!!!
iter  43<: x_1_d= 0.725; f_d= 2.650; jac_d= 0.239; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.874; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuis

...

iter  18>: x_13_d= 0.987; f_d= 2.738; jac_d= 0.368; nsteps= 3; x_d= 0.001; f_impr=-0.001; jac_impr= 0.283; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
-> iter  18>: !!!found NEW MAXIMUM for x_13 of  8.838 (+ 0.817) at [-1.95513922 -1.77974741  0.12515258  0.49030767  0.80360013  0.86589893
  0.63228357  2.19461803  0.83968367  1.34732942 -0.32077746 -0.74857111
  0.57860857  2.20285596]!!!
iter  19>: x_13_d= 0.987; f_d= 2.738; jac_d= 0.490; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.412; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
-> iter  19>: !!!found NEW MAXIMUM for x_13 of  8.839 (+ 0.818) at [-1.95462355 -1.77922852  0.12390929  0.48968937  0.8034692   0.8642432
  0.63418884  2.19488442  0.83996506  1.34718327 -0.32026415 -0.74865922
  0.57862871  2.20334424]!!!
iter  20>: x_13_d= 0.988; f_d= 2.740; jac_d= 0.382; nsteps= 3; x_d= 0.001; f_impr=-0.001; jac_impr= 0.361; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing 

iter  38>: x_13_d= 0.995; f_d= 2.747; jac_d= 0.172; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr= 0.373; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
-> iter  38>: !!!found NEW MAXIMUM for x_13 of  8.847 (+ 0.826) at [-1.9481697  -1.77339371  0.11614908  0.48308511  0.79883238  0.85423815
  0.62476182  2.19923247  0.84423651  1.35182364 -0.31111335 -0.74941368
  0.58079429  2.21103467]!!!
iter  39>: x_13_d= 0.995; f_d= 2.747; jac_d= 0.258; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr=-0.584; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
-> iter  39>: !!!found NEW MAXIMUM for x_13 of  8.847 (+ 0.826) at [-1.94791495 -1.77315169  0.11550026  0.48263497  0.79844092  0.85389918
  0.62566773  2.19945369  0.84450814  1.35176776 -0.31051663 -0.74947711
  0.58108932  2.21127881]!!!
iter  40>: x_13_d= 0.995; f_d= 2.747; jac_d= 0.151; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr= 0.775; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing

iter  79>: x_13_d= 2.032; f_d= 2.599; jac_d= 0.454; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.453; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter  80>: x_13_d= 2.032; f_d= 2.600; jac_d= 0.334; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.444; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter  81>: x_13_d= 2.033; f_d= 2.600; jac_d= 0.458; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.455; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter  82>: x_13_d= 2.033; f_d= 2.601; jac_d= 0.333; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.456; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter  83>: x_13_d= 2.034; f_d= 2.601; jac_d= 0.457; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.454; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter  84>: x_13_d= 2.034; f_d= 2.602; jac_d= 0.337; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.438

iter 125>: x_13_d= 2.411; f_d= 2.479; jac_d= 0.264; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr=-0.549; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 126>: x_13_d= 2.412; f_d= 2.480; jac_d= 0.172; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.617; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 127>: x_13_d= 2.412; f_d= 2.480; jac_d= 0.177; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr=-0.045; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
iter 128>: x_13_d= 2.413; f_d= 2.480; jac_d= 0.231; nsteps= 4; x_d= 0.001; f_impr=-0.000; jac_impr=-0.380; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 129>: x_13_d= 2.413; f_d= 2.481; jac_d= 0.219; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr= 0.076; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
iter 130>: x_13_d= 2.413; f_d= 2.481; jac_d= 0.171; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr= 0.357

iter 171>: x_13_d= 2.822; f_d= 2.401; jac_d= 0.178; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr=-0.053; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
iter 172>: x_13_d= 2.823; f_d= 2.401; jac_d= 0.222; nsteps= 4; x_d= 0.001; f_impr=-0.000; jac_impr=-0.320; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 173>: x_13_d= 2.823; f_d= 2.401; jac_d= 0.206; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.109; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 174>: x_13_d= 2.824; f_d= 2.401; jac_d= 0.149; nsteps= 3; x_d= 0.001; f_impr=-0.000; jac_impr= 0.470; f_e= 0.000; step= 0.000; radius= 0.001 - maximizing nuisance parameters
iter 175>: x_13_d= 2.824; f_d= 2.402; jac_d= 0.168; nsteps= 3; x_d= 0.000; f_impr=-0.000; jac_impr=-0.177; f_e= 0.000; step= 0.000; radius= 0.000 - maximizing nuisance parameters
iter 176>: x_13_d= 2.824; f_d= 2.402; jac_d= 0.196; nsteps= 4; x_d= 0.001; f_impr=-0.000; jac_impr=-0.221

In [112]:
imp.reload(CovidEM2) #applies changes made to file
np.random.seed(451)

bad_params = [-1.72245253, -0.91245638,  0.01888201,  0.34871368,  0.35877798,
              0.39279521,  0.41702544,  1.95818931, -0.10753684,  0.3775735 ,
              -9.22278978,  1.15643563,  1.2910407 ,  1.64917554]

CovidEM2.Covid_KL2(bad_params, prem, cases_dict, countries)

res = optimize.basinhopping(CovidEM2.Covid_KL2, bad_params, minimizer_kwargs = kwargs)

In [113]:
print(res)

                        fun: 0.3167849972770691
 lowest_optimization_result:       fun: 0.3167849972770691
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.55111516e-06,  5.00710582e-06, -6.49480470e-07, -1.16684440e-05,
        4.81281681e-06, -2.31481501e-06,  3.68594042e-06, -4.62407893e-06,
        1.40998323e-06, -1.06747944e-05, -1.11022293e-08, -3.96349618e-06,
       -1.32116539e-06,  7.93809467e-06])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1635
      nit: 99
     njev: 109
   status: 0
  success: True
        x: array([ -1.61258473,  -0.7744435 ,   0.09740116,   0.4341267 ,
         0.4574897 ,   0.49643494,   0.54340821,   1.54197721,
        -0.91237976,   0.01405923, -13.1434035 ,   0.72507274,
         0.92642529,   1.21675698])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 151485
                   

In [114]:
thetas = best_est
confint = cis_2b

print("Overhauled KL, Estimate")
print("non-trans: theta: ", thetas[0])
print("non-trans: CIs:", confint[0,:])
print("non-trans: theta - upper: ", thetas[0] - confint[0,0])
print("non-trans: theta - lower: ", thetas[0] - confint[0,1])
print("\n")

for i in range(0,len(thetas)):
    print("Parameter #", i+1)
    print("trans: theta: ", CovidEM2.param_trans2(thetas)[i])
    print("trans: CIs:", CovidEM2.param_trans2(confint[i,:]))
    print("trans: theta - upper:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,0]))
    print("trans: theta - lower:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,1]))
    print("\n")

Overhauled KL, Estimate
non-trans: theta:  -1.517340579207497
non-trans: CIs: [-1.75830738 -1.3460149 ]
non-trans: theta - upper:  0.240966796875
non-trans: theta - lower:  -0.17132568359375


Parameter # 1
trans: theta:  0.219294308
trans: CIs: [0.17233631766572943, 0.2602754213875373]
trans: theta - upper: 0.04695799033427056
trans: theta - lower: -0.040981113387537293


Parameter # 2
trans: theta:  0.423309799
trans: CIs: [0.3537965293739492, 0.6038150065919994]
trans: theta - upper: 0.06951326962605081
trans: theta - lower: -0.1805052075919994


Parameter # 3
trans: theta:  1.124597388
trans: CIs: [1.0342586189947878, 1.2918188582038286]
trans: theta - upper: 0.0903387690052122
trans: theta - lower: -0.1672214702038286


Parameter # 4
trans: theta:  1.511734653
trans: CIs: [1.3934404525236117, 1.787712020316659]
trans: theta - upper: 0.11829420047638828
trans: theta - lower: -0.275977367316659


Parameter # 5
trans: theta:  1.560386081
trans: CIs: [1.4334647434051875, 1.84563932395

In [ ]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          CovidEM2.param_trans2(cis_2b[:,0]),
          CovidEM2.param_trans2(best_est),
          CovidEM2.param_trans2(cis_2b[:,1])]
print(best_est - cis_2b[:,0])
print(best_est)
print(best_est - cis_2b[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)
